In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from data import *
from parameters import *
from regression import *

In [2]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(compute_percentile_threshold=0)
frontier_pcd_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Batch size,Batch size notes,Organization categorization,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Code accessibility,Dataset accessibility,Accessibility notes
39,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Gemini Team,SOTA improvement,""" Evaluation on a broad range of benchmarks sh...",Hosted access (no API),https://storage.googleapis.com/deepmind-media/...,252.0,Gemini: A Family of Highly Capable Multimodal ...,...,NaN,NaN,Industry,NaN,NaN,NaN,132000000.0,NaN,NaN,NaN
46,Qwen-72B,Language,"Chat,Code generation","Jinze Bai, Shuai Bai, Yunfei Chu, Zeyu Cui, Ka...",SOTA improvement,"SOTA on several Chinese benchmarks, with highe...",Permissive license (depr.),https://huggingface.co/Qwen/Qwen-72B,NaN,NaN,...,4000000.0,Table 1 https://arxiv.org/abs/2309.16609\n(thi...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Inflection-2,Language,Language modelling,NaN,Significant use,Inflection-2 either already powers Pi or soon ...,API access,https://inflection.ai/inflection-2,NaN,Inflection-2: The Next Step Up,...,NaN,NaN,Industry,checked,NaN,NaN,NaN,NaN,NaN,NaN
59,Nemotron-3-8B,Language,"Chat,Language generation",NaN,SOTA improvement,"""The Nemotron-3-8B-QA model offers state-of-th...",Permissive license (depr.),https://developer.nvidia.com/blog/nvidia-ai-fo...,NaN,NVIDIA AI Foundation Models: Build Custom Ente...,...,NaN,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,MultiBand Diffusion,Audio,Audio generation,"Robin San Roman, Yossi Adi, Antoine Deleforge,...",SOTA improvement,"""At equal bit rate, the proposed approach outp...",Permissive license (depr.),https://arxiv.org/abs/2308.02560,1.0,From Discrete Tokens to High-Fidelity Audio Us...,...,NaN,NaN,"Industry,Academia,Academia",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14733.0,Mastering the game of Go with deep neural netw...,...,NaN,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1132,"Variational (untied weights, MC) LSTM (Large)",Language,NaN,"Yarin Gal, Zoubin Ghahramani","Highly cited,SOTA improvement","""The new approach outperforms existing techniq...",NaN,https://arxiv.org/abs/1512.05287?context=stat,1838.0,A Theoretically Grounded Application of Dropou...,...,NaN,NaN,Academia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1135,ResNet-152 (ImageNet),Vision,Image classification,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.03385,154061.0,Deep Residual Learning for Image Recognition,...,NaN,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1137,DeepSpeech2 (English),Speech,Speech recognition,"Dario Amodei, Rishita Anubhai, Eric Battenberg...",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.02595,2741.0,Deep Speech 2: End-to-End Speech Recognition i...,...,NaN,NaN,Industry,NaN,NaN,NaN,301.0,NaN,NaN,NaN


In [3]:
hardware_aliases = ['A100', 'H100', 'P100', 'V100', 'TPU v4', 'TPU v3', 'TPU v2', 'TPU v1', 'K80', 'K40']
hardware_counts = defaultdict(int)
for hardware in frontier_pcd_df['Training hardware'].dropna():
    alias_found = False
    for alias in hardware_aliases:
        if alias in hardware:
            hardware_counts[alias] += 1
            alias_found = True
    if not alias_found:
        hardware_counts[hardware] += 1

# Print counts in descending order
for hardware, count in sorted(hardware_counts.items(), key=lambda x: x[1], reverse=True):
    print(f'{hardware}: {count}')

A100: 50
V100: 49
TPU v3: 38
TPU v4: 19
TPU v2: 4
P100: 3
K80: 3
NVIDIA GTX Titan X: 3
NVIDIA Geforce GTX1080 Ti: 2
K40: 2
TPU v1: 2
NVIDIA GeForce GTX TITAN X: 2
H100: 1
NVIDIA A800: 1
Huawei Ascend 910: 1
NVIDIA Quadro RTX 4000: 1
NVIDIA RTX A6000: 1
NVIDIA Quadro RTX 8000: 1
NVIDIA Quadro RTX 5000: 1
NVIDIA GeForce RTX 2080 Ti: 1
NVIDIA TITAN Xp: 1
NVIDIA Quadro P600: 1
NVIDIA M40,NVIDIA GTX Titan X: 1
NVIDIA M40: 1


In [4]:
# No TPUs
hardware_aliases = ['A100', 'H100', 'P100', 'V100', 'TPU v4', 'TPU v3', 'TPU v2', 'TPU v1', 'K80', 'K40']
hardware_counts = defaultdict(int)
for hardware in frontier_pcd_df['Training hardware'].dropna():
    if 'TPU' in hardware:
        continue
    alias_found = False
    for alias in hardware_aliases:
        if alias in hardware:
            hardware_counts[alias] += 1
            alias_found = True
    if not alias_found:
        hardware_counts[hardware] += 1

# Print counts in descending order
for hardware, count in sorted(hardware_counts.items(), key=lambda x: x[1], reverse=True):
    print(f'{hardware}: {count}')

A100: 50
V100: 49
P100: 3
K80: 3
NVIDIA GTX Titan X: 3
NVIDIA Geforce GTX1080 Ti: 2
K40: 2
NVIDIA GeForce GTX TITAN X: 2
H100: 1
NVIDIA A800: 1
Huawei Ascend 910: 1
NVIDIA Quadro RTX 4000: 1
NVIDIA RTX A6000: 1
NVIDIA Quadro RTX 8000: 1
NVIDIA Quadro RTX 5000: 1
NVIDIA GeForce RTX 2080 Ti: 1
NVIDIA TITAN Xp: 1
NVIDIA Quadro P600: 1
NVIDIA M40,NVIDIA GTX Titan X: 1
NVIDIA M40: 1


# Purchase prices

In [5]:
price_colname = 'Price (hardware purchase)'
purchase_price_df = price_df.dropna(subset=[price_colname])
purchase_price_df

,Price source,Price date,Hardware model,Manufacturer (from Hardware model),Vendor,Location,Price per chip-hour (on-demand),Price per chip-hour (1-year CUD),Price per chip-hour (3-year CUD),Price (hardware purchase),Notes
25,https://www.nextplatform.com/2022/05/09/how-mu...,2020-01-01,NVIDIA A100 PCIe,NVIDIA,NaN,NaN,NaN,NaN,NaN,15000.00,Single-unit list price
26,https://www.nextplatform.com/2022/05/09/how-mu...,2022-01-01,NVIDIA A100 PCIe,NVIDIA,NaN,NaN,NaN,NaN,NaN,12500.00,Single-unit list price
38,https://www.nextplatform.com/2022/05/09/how-mu...,2020-01-01,NVIDIA A100 SXM4 40 GB,NVIDIA,NaN,NaN,NaN,NaN,NaN,15000.00,Single-unit list price
39,https://web.archive.org/web/20200521074015/htt...,2020-05-21,NVIDIA A100 SXM4 40 GB,NVIDIA,NaN,NaN,NaN,NaN,NaN,24875.00,"DGX A100, 8 GPU, 320GB. Release price."
40,https://www.nextplatform.com/2022/05/09/how-mu...,2022-01-01,NVIDIA A100 SXM4 40 GB,NVIDIA,NaN,NaN,NaN,NaN,NaN,12500.00,Single-unit list price
48,https://web.archive.org/web/20210630170623/htt...,2021-06-30,NVIDIA A100 SXM4 80 GB,NVIDIA,NaN,NaN,NaN,NaN,NaN,37500.00,"DGX A100, 8 GPU, 640GB"
49,https://web.archive.org/web/20220120191032/htt...,2022-01-20,NVIDIA A100 SXM4 80 GB,NVIDIA,NaN,NaN,NaN,NaN,NaN,20875.00,"DGX A100, 8 GPU, 640GB"
50,https://web.archive.org/web/20230923154035/htt...,2023-09-23,NVIDIA A100 SXM4 80 GB,NVIDIA,NaN,NaN,NaN,NaN,NaN,18548.75,"HGX A100, 8 GPU, 640GB"
57,https://www.techpowerup.com/gpu-specs/geforce-...,2015-03-17,NVIDIA GTX Titan X,NVIDIA,NaN,NaN,NaN,NaN,NaN,999.00,Single-unit release price
59,https://web.archive.org/web/20220929115124/htt...,2022-09-29,NVIDIA H100 SXM5,NVIDIA,NaN,NaN,NaN,NaN,NaN,44489.88,"DGX, 8 GPU, 640GB. Convert Euro to USD: https:..."


In [6]:
# Multiply single unit prices by overhead factor
server_adjusted_purchase_price_df = pd.DataFrame(columns=purchase_price_df.columns)
for i, row in purchase_price_df.iterrows():
    if 'single-unit' in row['Notes'].lower():
        row[price_colname] *= SERVER_COST_OVERHEAD
        row['Notes'] = row['Notes'].lower().replace('single-unit', 'server-equivalent')
    server_adjusted_purchase_price_df.loc[i] = row
server_adjusted_purchase_price_df

,Price source,Price date,Hardware model,Manufacturer (from Hardware model),Vendor,Location,Price per chip-hour (on-demand),Price per chip-hour (1-year CUD),Price per chip-hour (3-year CUD),Price (hardware purchase),Notes
25,https://www.nextplatform.com/2022/05/09/how-mu...,2020-01-01,NVIDIA A100 PCIe,NVIDIA,NaN,NaN,NaN,NaN,NaN,24000.0,server-equivalent list price
26,https://www.nextplatform.com/2022/05/09/how-mu...,2022-01-01,NVIDIA A100 PCIe,NVIDIA,NaN,NaN,NaN,NaN,NaN,20000.0,server-equivalent list price
38,https://www.nextplatform.com/2022/05/09/how-mu...,2020-01-01,NVIDIA A100 SXM4 40 GB,NVIDIA,NaN,NaN,NaN,NaN,NaN,24000.0,server-equivalent list price
39,https://web.archive.org/web/20200521074015/htt...,2020-05-21,NVIDIA A100 SXM4 40 GB,NVIDIA,NaN,NaN,NaN,NaN,NaN,24875.0,"DGX A100, 8 GPU, 320GB. Release price."
40,https://www.nextplatform.com/2022/05/09/how-mu...,2022-01-01,NVIDIA A100 SXM4 40 GB,NVIDIA,NaN,NaN,NaN,NaN,NaN,20000.0,server-equivalent list price
48,https://web.archive.org/web/20210630170623/htt...,2021-06-30,NVIDIA A100 SXM4 80 GB,NVIDIA,NaN,NaN,NaN,NaN,NaN,37500.0,"DGX A100, 8 GPU, 640GB"
49,https://web.archive.org/web/20220120191032/htt...,2022-01-20,NVIDIA A100 SXM4 80 GB,NVIDIA,NaN,NaN,NaN,NaN,NaN,20875.0,"DGX A100, 8 GPU, 640GB"
50,https://web.archive.org/web/20230923154035/htt...,2023-09-23,NVIDIA A100 SXM4 80 GB,NVIDIA,NaN,NaN,NaN,NaN,NaN,18548.75,"HGX A100, 8 GPU, 640GB"
57,https://www.techpowerup.com/gpu-specs/geforce-...,2015-03-17,NVIDIA GTX Titan X,NVIDIA,NaN,NaN,NaN,NaN,NaN,1598.4,server-equivalent release price
59,https://web.archive.org/web/20220929115124/htt...,2022-09-29,NVIDIA H100 SXM5,NVIDIA,NaN,NaN,NaN,NaN,NaN,44489.88,"DGX, 8 GPU, 640GB. Convert Euro to USD: https:..."


In [7]:
fig = go.Figure()
for hardware_alias in hardware_aliases:
    hardware_df = server_adjusted_purchase_price_df[server_adjusted_purchase_price_df['Hardware model'].str.contains(hardware_alias)]
    fig.add_trace(go.Scatter(
        x=hardware_df['Price date'],
        y=hardware_df[price_colname],
        mode='markers',
        name=hardware_alias,
        text=server_adjusted_purchase_price_df['Hardware model'] + ' | ' + hardware_df['Notes'],
    ))
fig.update_layout(
    title='Server-equivalent purchase price vs. performance',
    xaxis_title='Price date',
    yaxis_title='Server-equivalent purchase price ($)',
    # yaxis_type='log',
    width=800,
    height=600,
)
fig.show()

In [8]:
# Drop thenextplatform rows
server_adjusted_purchase_price_df = server_adjusted_purchase_price_df[~server_adjusted_purchase_price_df['Price source'].str.contains('nextplatform')]

In [9]:
fig = go.Figure()
for hardware_alias in hardware_aliases:
    hardware_df = server_adjusted_purchase_price_df[server_adjusted_purchase_price_df['Hardware model'].str.contains(hardware_alias)]
    fig.add_trace(go.Scatter(
        x=hardware_df['Price date'],
        y=hardware_df[price_colname],
        mode='markers',
        name=hardware_alias,
        text=server_adjusted_purchase_price_df['Hardware model'] + ' | ' + hardware_df['Notes'],
    ))
fig.update_layout(
    title='Server-equivalent purchase price vs. performance',
    xaxis_title='Price date',
    yaxis_title='Server-equivalent purchase price ($)',
    # yaxis_type='log',
    width=800,
    height=600,
)
fig.show()

# Training time

In [10]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(compute_percentile_threshold=75)
frontier_pcd_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Batch size,Batch size notes,Organization categorization,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Code accessibility,Dataset accessibility,Accessibility notes
39,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Gemini Team,SOTA improvement,""" Evaluation on a broad range of benchmarks sh...",Hosted access (no API),https://storage.googleapis.com/deepmind-media/...,252.0,Gemini: A Family of Highly Capable Multimodal ...,...,NaN,NaN,Industry,NaN,NaN,NaN,132000000.0,NaN,NaN,NaN
46,Qwen-72B,Language,"Chat,Code generation","Jinze Bai, Shuai Bai, Yunfei Chu, Zeyu Cui, Ka...",SOTA improvement,"SOTA on several Chinese benchmarks, with highe...",Permissive license (depr.),https://huggingface.co/Qwen/Qwen-72B,NaN,NaN,...,4000000.0,Table 1 https://arxiv.org/abs/2309.16609\n(thi...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Inflection-2,Language,Language modelling,NaN,Significant use,Inflection-2 either already powers Pi or soon ...,API access,https://inflection.ai/inflection-2,NaN,Inflection-2: The Next Step Up,...,NaN,NaN,Industry,checked,NaN,NaN,NaN,NaN,NaN,NaN
83,Yi-34B,Language,Chat,NaN,Significant use,2nd most popular model on HuggingFace: https:/...,Permissive license (depr.),https://arxiv.org/abs/2403.04652,NaN,NaN,...,NaN,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,ChatGLM3,Multimodal,"Chat,Visual question answering",NaN,SOTA improvement,"Aiming at GPT-4V, ChatGLM3 has implemented ite...",NaN,https://www.zhipuai.cn/en/news/76,NaN,Zhipu AI launches third-generation base model,...,NaN,NaN,Industry,checked,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,Xception,Vision,Image classification,François Chollet,Highly cited,NaN,NaN,https://arxiv.org/abs/1610.02357,11338.0,Xception: Deep Learning with Depthwise Separab...,...,NaN,NaN,Industry,NaN,NaN,NaN,43200.0,NaN,NaN,NaN
1095,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,NaN,https://arxiv.org/abs/1609.08144,6105.0,Google's Neural Machine Translation System: Br...,...,NaN,NaN,Industry,NaN,NaN,NaN,414720.0,NaN,NaN,NaN
1131,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14733.0,Mastering the game of Go with deep neural netw...,...,NaN,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1137,DeepSpeech2 (English),Speech,Speech recognition,"Dario Amodei, Rishita Anubhai, Eric Battenberg...",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.02595,2741.0,Deep Speech 2: End-to-End Speech Recognition i...,...,NaN,NaN,Industry,NaN,NaN,NaN,301.0,NaN,NaN,NaN


In [11]:
training_time_df = frontier_pcd_df.dropna(subset=['Training time (hours)'])
training_time_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Batch size,Batch size notes,Organization categorization,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Code accessibility,Dataset accessibility,Accessibility notes
39,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Gemini Team,SOTA improvement,""" Evaluation on a broad range of benchmarks sh...",Hosted access (no API),https://storage.googleapis.com/deepmind-media/...,252.0,Gemini: A Family of Highly Capable Multimodal ...,...,NaN,NaN,Industry,NaN,NaN,NaN,132000000.0,NaN,NaN,NaN
135,Falcon 180B,Language,Language modelling,"Ebtesam Almazrouei, Hamza Alobeidli, Abdulaziz...",SOTA improvement,"""It's currently at the top of the Hugging Face...",Permissive license (depr.),https://falconllm.tii.ae/falcon-180b.html; htt...,39.0,Falcon LLM - Falcon 180B,...,4194304.0,from paper (https://arxiv.org/pdf/2311.16867.p...,Government,NaN,NaN,NaN,17694720.0,NaN,NaN,NaN
174,Llama 2-70B,Language,Language modelling,"Hugo Touvron, Louis Martin, Kevin Stone, Peter...","Historical significance,Significant use,Highly...",Model has been open-sourced and frequently dow...,Fully open-source (depr.),https://ai.meta.com/research/publications/llam...,2507.0,Llama 2: Open Foundation and Fine-Tuned Chat M...,...,4000000.0,NaN,Industry,NaN,NaN,NaN,2160000.0,NaN,NaN,NaN
186,xTrimoPGLM -100B,Biology,Proteins,"Bo Chen, Xingyi Cheng, Yangli-ao Geng, Shen Li...",SOTA improvement,"""Our extensive experiments reveal that xTrimoP...",Weights available (depr.),https://www.biorxiv.org/content/10.1101/2023.0...,23.0,xTrimoPGLM: Unified 100B-Scale Pre-trained Tra...,...,NaN,NaN,"Academia,Industry",NaN,NaN,NaN,2352.0,NaN,NaN,NaN
262,PanGu-Σ,Language,"Code generation,Language modelling","Xiaozhe Ren, Pingyi Zhou, Xinfan Meng, Xinjing...",SOTA improvement,"""Our experimental findings show that PanGu-{\S...",NaN,https://arxiv.org/abs/2303.10845,34.0,PanGu-Σ: Towards Trillion Parameter Language M...,...,524288.0,"""We train PanGu-Σ with global batch size of 51...",Industry,NaN,NaN,NaN,1228800.0,NaN,NaN,NaN
263,GPT-4,Multimodal,Language modelling,OpenAI,"Highly cited,SOTA improvement","See the paper, p.1: ""On a suite of traditional...",API access,https://arxiv.org/abs/2303.08774,2767.0,GPT-4 Technical Report,...,NaN,NaN,Industry,checked,NaN,NaN,57000000.0,NaN,NaN,NaN
279,LLaMA-65B,Language,Language modelling,"Hugo Touvron, Thibaut Lavril, Gautier Izacard,...","Historical significance,Highly cited",Widely-used foundation model that has been ada...,Fully open-source (depr.),https://arxiv.org/abs/2302.13971,4055.0,LLaMA: Open and Efficient Foundation Language ...,...,4000000.0,NaN,Industry,checked,NaN,NaN,1024000.0,NaN,NaN,NaN
334,BLOOM-176B,Language,Language modelling,"Margaret Mitchell, Giada Pistilli, Yacine Jern...","Historical significance,Highly cited",Was the largest open-source model at the time....,Permissive license (depr.),https://arxiv.org/abs/2211.05100,1002.0,BLOOM: A 176B-Parameter Open-Access Multilingu...,...,4194304.0,Table 3. 2048*2048,"Industry,Research collective",checked,NaN,NaN,1078272.0,NaN,NaN,NaN
349,U-PaLM (540B),Language,Language generation,"Yi Tay, Jason Wei, Hyung Won Chung, Vinh Q. Tr...",SOTA improvement,"""We show that U-PaLM 540B outperforms PaLM 540...",Unreleased,https://arxiv.org/abs/2210.11399,43.0,Transcending Scaling Laws with 0.1% Extra Compute,...,NaN,NaN,Industry,NaN,NaN,NaN,61440.0,NaN,NaN,NaN
375,GLM-130B,Language,NaN,NaN,SOTA improvement,"""GLM-130B achieves an accuracy of 80.2% on zer...",NaN,https://keg.cs.tsinghua.edu.cn/glm-130b/posts/...,60.0,GLM-130B: An open bilingual pre-trained model,...,NaN,NaN,Academia,checked,NaN,NaN,1105920.0,NaN,NaN,NaN


In [12]:
training_time_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(training_time_df['Publication date']))
reg_results = fit_ols_regression(training_time_df, ['Publication date (float)'], 'Training time (hours)', logy=True)
reg_results.summary()

/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipykernel_68070/1688430577.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     6.884
Date:                Fri, 03 May 2024   Prob (F-statistic):             0.0124
Time:                        13:57:28   Log-Likelihood:                -26.292
No. Observations:                  41   AIC:                             56.58
Df Residuals:                      39   BIC:                             60.01
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -192.2298     74.366     -2.585      0.014    -342.648     -41.811
x1             0.0965      0.037      2.624      0.012       0.022       0.171
==============================================================================
Omnibus:                        0.872   Durbin-Watson:                   1.340
Prob(Omnibus):                  0.647   Jarque-Bera (JB):                0.231
Skew:                           0.092   Prob(JB):                        0.891
Kurtosis:                       3.318   Cond. No.                     2.04e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.04e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
# Plot training time
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=training_time_df['Publication date'],
    y=training_time_df['Training time (hours)'],
    mode='markers',
    text=training_time_df['System']
))
fig.update_layout(
    title='Training times of notable models',
    xaxis_title='Publication date',
    yaxis_title='Training time (hours)',
    yaxis_type='log',
    width=800,
    height=600,
)
fig.show()

In [14]:
np.median(training_time_df['Training time (hours)'])

770.0

In [15]:
np.percentile(training_time_df['Training time (hours)'], 5)

100.0

In [16]:
np.percentile(training_time_df['Training time (hours)'], 95)

4320.0

In [23]:
start_price = 20000
cost = 0
chip_days = 25000 * 90
for i in range(chip_days):
    cost += start_price * (1 - 0.1**(ML_GPU_PRICE_PERFORMANCE_OOMS_PER_YEAR / DAYS_PER_YEAR))
cost

39698525.66383674

39698525.66383674

In [28]:
a = 20000
r = 10**(-ML_GPU_PRICE_PERFORMANCE_OOMS_PER_YEAR / DAYS_PER_YEAR)
n = 90
cost = 25000 * a * (1 - r**n) / (1 - r)
cost

43278266482.312256

In [31]:
1 - r

0.0008821894591714008